In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import panel as pn

pn.extension('plotly')

# Load the dataset
df = pd.read_csv('Spring_2025_schedule_standardized.csv')

# Clean and preprocess
df = df[df['MAXCAPACITY'] > 0]
df['Utilization'] = df['SECTENROLL'] / df['MAXENROLL']
df = df[~df['CAMPUS'].isin(['GU', 'HS'])]
df = df[df['CRSSTATUS'].str.lower() == 'active']
df = df[df['INST_METHOD'].isin(['TD', 'BLD'])]
df = df[df['MTGDAYS'].notna()]
df = df[~df['MTGDAYS'].str.upper().eq('TBA')]
df = df[~df['PARTTERM'].isin(['K', 'HS'])]

# Create dynamic widgets
campus_selector = pn.widgets.Select(name='Select Campus', options=['DF', 'MT', 'BX'], value='DF')
partterm_options = sorted(df['PARTTERM'].unique())
partterm_selector = pn.widgets.CheckBoxGroup(name='PARTTERM Filter', value=partterm_options, options=partterm_options)

@pn.depends(campus_selector, partterm_selector)
def update_plot(selected_campus, selected_partterms):
    filtered_df = df[(df['CAMPUS'] == selected_campus) & (df['PARTTERM'].isin(selected_partterms))]

    if filtered_df.empty:
        return pn.pane.Markdown("### ⚠️ No data available for the selected filters.")

    avg_util = filtered_df['Utilization'].mean()

    plt.figure(figsize=(12, 6))
    sns.boxplot(x='ROOM', y='Utilization', data=filtered_df, palette='pastel')
    plt.title(f'Spring 2025 Room Utilization in {selected_campus}', fontsize=16)
    plt.ylabel('Utilization Ratio (Enrolled / Capacity)', fontsize=12)
    plt.xlabel('Room', fontsize=12)
    plt.xticks(rotation=90)
    plt.axhline(avg_util, color='red', linestyle='--', linewidth=1.5, label=f'Average: {avg_util:.2f}')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()

    return pn.pane.Matplotlib(plt.gcf(), tight=True)

# Display Panel layout
panel_layout = pn.Column(
    "## 🏫 Room Utilization Dashboard",
    pn.Row(campus_selector, partterm_selector),
    update_plot
)

panel_layout.servable()
